In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/Colab\ Notebooks/NeuroImage/UKB_Paper_Example_Git/ML-approach

/content/drive/My Drive/Colab Notebooks/NeuroImage/UKB_Paper_Example_Git/ML-approach


# **STEP 0:** File/Package Loading

In [3]:
import preparation as prep #class needed to create the split using metadata file
import linear_model as cl #to train the (un)penalized linear models-based brain clocks
import test_metrics_all as tm #to compute performance metrics in test set
import corrector as cr #to correct models not clearing t-test/p-values for all coefficients
import os
import shutil
import pickle
import pandas as pd
import json

**Observation** (Display input metadata): To comply with MTA agreements, we have added noise to all numerical columns and altered all available dates, including birth dates, visit dates, diagnosis dates, and death dates.

In [4]:
data = pd.read_csv('perturbed_data.csv')
pd.set_option('display.max_columns', None)
data

,unique_id,Left-Cerebral-White-Matter,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,4th-Ventricle,Brain-Stem,Left-Hippocampus,Left-Amygdala,CSF,Left-Accumbens-area,Left-VentralDC,Left-choroid-plexus,Right-Cerebral-White-Matter,Right-Lateral-Ventricle,Right-Inf-Lat-Vent,Right-Cerebellum-White-Matter,Right-Cerebellum-Cortex,Right-Thalamus,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,Right-choroid-plexus,WM-hypointensities,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior,ctx-lh-caudalanteriorcingulate,ctx-lh-caudalmiddlefrontal,ctx-lh-cuneus,ctx-lh-entorhinal,ctx-lh-fusiform,ctx-lh-inferiorparietal,ctx-lh-inferiortemporal,ctx-lh-isthmuscingulate,ctx-lh-lateraloccipital,ctx-lh-lateralorbitofrontal,ctx-lh-lingual,ctx-lh-medialorbitofrontal,ctx-lh-middletemporal,ctx-lh-parahippocampal,ctx-lh-paracentral,ctx-lh-parsopercularis,ctx-lh-parsorbitalis,ctx-lh-parstriangularis,ctx-lh-pericalcarine,ctx-lh-postcentral,ctx-lh-posteriorcingulate,ctx-lh-precentral,ctx-lh-precuneus,ctx-lh-rostralanteriorcingulate,ctx-lh-rostralmiddlefrontal,ctx-lh-superiorfrontal,ctx-lh-superiorparietal,ctx-lh-superiortemporal,ctx-lh-supramarginal,ctx-lh-transversetemporal,ctx-lh-insula,ctx-rh-caudalanteriorcingulate,ctx-rh-caudalmiddlefrontal,ctx-rh-cuneus,ctx-rh-entorhinal,ctx-rh-fusiform,ctx-rh-inferiorparietal,ctx-rh-inferiortemporal,ctx-rh-isthmuscingulate,ctx-rh-lateraloccipital,ctx-rh-lateralorbitofrontal,ctx-rh-lingual,ctx-rh-medialorbitofrontal,ctx-rh-middletemporal,ctx-rh-parahippocampal,ctx-rh-paracentral,ctx-rh-parsopercularis,ctx-rh-parsorbitalis,ctx-rh-parstriangularis,ctx-rh-pericalcarine,ctx-rh-postcentral,ctx-rh-posteriorcingulate,ctx-rh-precentral,ctx-rh-precuneus,ctx-rh-rostralanteriorcingulate,ctx-rh-rostralmiddlefrontal,ctx-rh-superiorfrontal,ctx-rh-superiorparietal,ctx-rh-superiortemporal,ctx-rh-supramarginal,ctx-rh-transversetemporal,ctx-rh-insula,db_name,acquisition_date,biological_sex,birth_date,month_birth,year_birth,education_years,ethnicity,cohort,visit_date,CDR,MMSE,MOCA,cognitive_status,etiology,Dementia,VascularDementia,OtherDiseaseDementia,UnspecifiedDementia,Huntington,PrimaryParkinson,SecondaryParkinson,Alzheimer,MultipleSclerosis,otherSpinalChord,dt,age_at_scan,unique_subj_id,machine_model,manufacturer,death_date,death_primary_cause,death_primary_cause_text,last_visit_date
0,dADNI_s00314_i002_r01,213040.078330,42744.317178,4059.591505,12990.628223,43315.713179,5805.641381,3358.098363,4771.819630,2067.673200,3074.592628,2086.492599,20961.522495,2797.020911,1076.827259,2016.260795,397.398682,3830.580149,1677.827575,218387.028883,31159.649075,3711.751342,11833.098596,43834.968299,6026.296580,3328.969653,4610.529421,2005.402065,3117.431506,1358.585318,464.625447,3765.941258,1704.434206,13376.143030,0.0,0.0,0.0,0.0,0.0,2479.564060,4582.546687,3382.980645,770.124482,5976.741535,9701.428320,8303.052292,2229.354435,9123.515025,6057.962661,5480.570085,3109.672798,9687.692626,1360.261374,3500.456827,2651.961376,1374.385253,2568.552923,1483.130567,6908.371713,2417.094870,11219.970571,6865.827671,2384.760258,8426.402644,18050.401183,9370.042937,9439.270295,6679.692330,897.777880,4423.986909,2060.567430,3507.723409,3303.744491,929.239834,6032.504640,9081.614463,8725.905898,1580.167603,9293.230577,6284.283380,5975.098506,3465.470976,10894.242932,1345.446394,3402.250041,3196.812809,1449.203598,2553.931174,1571.476327,7251.064234,3040.983721,9999.547130,7040.739572,2019.966125,10290.779420,18954.696835,8244.666669,9625.854301,7013.742572,535.160270,4670.690623,ADNI,2019-06-17,Male,1937-11-01,9.0,1937.0,16.0,White,AD,2019-08-17,8.0,21.0,18.0,Dementia,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,81.833333,dADNI_s00314,Skyra,Siemens,NaN,NaN,NaN,2020-11-18
1,dADNI_s00314_i003_r01,209097.976771,45887.280357,4252.390680,12366.456035,42755.442201

# **STEP 1:** Preparation Prior Training

**1.1.** Configuration of metadata splitting setting parameters

In [4]:
"""
:param age_limits: Age interval in years (list: [a,b]) denoting the age bracket considered during training/testing of developed models. Default: a=55,b=85.
:param delta_exclusion_months: MRI delta interval/list in months to be used only as test set, (a,b]. Default: a=2, b=4.
:param stratified_by_CN: Whether or not CN_type must be considered in stratified splitting. If not, just consider 'healthy_orx'. Default: True.
:param time_splits: Time splits to be considered to generate CN_type. Default: [0,5,10].
:param correct_pvals: Whether or not to correct models failing p-values (default p-val threshold: 0.05) and amount of repetitions. If 0, it is not performed,
        if more than 0, repeated as many times as correct_pvals. Default: 1.
:param run_id: Execution id number which is used to look into corresponding history of settings combinations explored/corrected file. Default: 1.
:param generate_test: Boolean, whether or not to compute test metrics. Default: True.
:param delete_existing_test: Boolean, whether or not to eliminate and redo full test analysis. Default: True
"""
age_limits = [55,85]
delta_exclusion_months = None
stratified_by_CN = True
time_splits = [0,5,10]
correct_pvals = 10
run_id = 1
generate_test = True
delete_existing_test = True

**1.2.** If requested delete preexisting tes metrics information

In [6]:
if delete_existing_test:
    file_path_register = 'historic_register_' + str(run_id) + '.json'
    if not os.path.exists(file_path_register): # If not .json file is available containing already explore brain clock modeling approaches, initiate one.
        examined_combinations = []
        corrected_ids = []
        data = {}
        id = 1
        data['examined_combinations'] = examined_combinations
        data['corrected_ids'] = corrected_ids
        data['tested'] = {}
        data['id'] = id
        with open(file_path_register, 'w') as file:
            json.dump(data, file)
    else: # If such a file exists, then eliminate information associated to pre-tested models.
        with open(file_path_register, 'r') as file:
            data = json.load(file)
        if len(data['tested'])>0:
            # Eliminate tested cases
            data['tested'] = {}
            with open(file_path_register, 'w') as file:
                json.dump(data, file)
            print(file_path_register + "file updated")
            # Eliminate test directories
            directories_test = ['metrics_test/', 'plots/', 'pareto/']
            for directory in directories_test:
                if os.path.exists(directory) and os.path.isdir(directory):
                    shutil.rmtree(directory)
                    print(f"Directory '{directory}' has been deleted.")
                else:
                    print(f"Directory '{directory}' does not exist.")

**1.3.** Check whether we already have a metadata split available for the requested population and settings to analyze. If not, generate a split (**input**: metadata displayed before, **output**: information collected in 'data/' folder, which contains the metadata split and various plots showing the obtained distributions, see .zip file shared with submission)

In [6]:
preprocessing_type = 'age_' + str(age_limits) + '_time_splits_' + str(time_splits) + '_delta_exclusion_' + str(delta_exclusion_months) #just for file naming

if os.path.isfile('data/' + 'train_data:' + preprocessing_type + '.csv'): # if split availble, extract info
    train_data = pd.read_csv('data/' + 'train_data:' + preprocessing_type + '.csv')
    test_data = pd.read_csv('data/' + 'test_data:' + preprocessing_type + '.csv')
    test_data['education_years'] = pd.cut(test_data['education_years'], bins=[0, 7, 15, 30], labels=['0-7', '8-15', '16-30'],
                                    include_lowest=True, right=True)
    with open('data/binary_cases' + preprocessing_type + '.pkl', 'rb') as file:
        full_set_of_predictors, binary_cases_train, binary_cases_test, binary_columns_train, binary_columns_test = pickle.load(file)

else: # otherwise, generate split and extract info
    dp = prep.dataset_selection(data_location = 'perturbed_data.csv',age_limits = age_limits,
                                time_splits = time_splits, delta_exclusion_months = delta_exclusion_months)
    train_data = dp.train_data
    test_data = dp.test_data
    full_set_of_predictors = dp.full_set_of_predictors
    binary_cases_train = dp.binary_cases_train
    binary_cases_test = dp.binary_cases_test
    binary_columns_train = dp.binary_columns_train
    binary_columns_test = dp.binary_columns_test
    preprocessing_type = dp.preprocessing_type

# **STEP 2:** Brain Clock Model Training

**2.1.** Predefine feature settings and type of models to be used to develop the brain clocks. If any of the settings has been already considered in a previous training phase, neglect.

In this notebook we consider a small subset of the models that have been used to reduce running times drastically (please refer to **main.py** to see the cases explored in the manuscript)

**input**: metadata split prepared in step 1.3, **output**: information collected in the 'metrics_train/' folder (training metrics and model metadata for all of the providedconfigurations) and 'models/' folder (directory where all of the models and associated age-bias correction parameters are stored). See .zip file shared with submission


In [ ]:
for T1W in [{'ctx_feats': 'Merged', 'LR_feats': 'Merged', 'CC_feats': True, 'extra_feats': True},
 {'ctx_feats': 'All', 'LR_feats': 'All', 'CC_feats': True, 'extra_feats': True}]: # dictionary with settings to be explored on FastSurfer features ('Merged' implies to add both hemispheres values, 'All', refers to keeping them separate).
    for training_population in ['orx','cole']: # healthy definition to be used
        for gender in ['All', 'Female']:# genders to be analyzed in modelling phase
            for high_pairwise_correlation_removal in [True]:# whether or not to remove highly correlated features:
                for method in ['ols','lasso']: # type of models to be considered
                    for sampling in [None, 'resample']:# sampling approaches to deal with age-bias.
                          model = cl.LMB(age_limits = age_limits, gender=gender, training_population = training_population,
                                          method = method, sampling = sampling, database_origin = 'data/' + 'train_data:' + preprocessing_type + '.csv',
                                          T1W = T1W,
                                          run_id = run_id, high_pairwise_correlation_removal = high_pairwise_correlation_removal)
                          if model.proceed:
                              model.fit(train_data)
                          else:
                              print('pre-executed case', model.combination)

**2.2.** Correct those models (OLS) failing the individualized coefficient t-test if requested by user (this step can repeated "correct_pvals" times in case newly developed models still faile to pass corresponding t-test)


In [ ]:
for i in range(correct_pvals):
    model = cr.retrain(X = train_data, run_id = run_id)
    model.models_to_correct()

# **STEP 3:** Brain Clock Model Test & Selection

**3.1.** Predefine test settings and type of models to be used to be considered in the analysis. If any of the settings has been already considered in a previous training phase, neglect if delete_existing_test True.

**input**: models developed in step 2 and test set extracted in step 1, **output**: information collected in the 'metrics_test/' folder (test metrics for all provided configurations), 'plots/' folder (directory where for all models considered in the analysis we show various plots over the test set to analyze brain age and PAD distribution on both healthy and diseased subpopulations) and 'pareto/' folder (where we do an initial selection of the most promising models based on the predefined performance metrics, pareto_feature_cases. Observe that in the paper a similar strategy is performed but following a stepwise approach on the external datasets, NACC and ADNI, which is described in the "comparison/" repo). See .zip file shared with submission


In [ ]:
if generate_test:
    for db in [None]:# database to be considered for model comparison. If None, then use all databases together
        for gender_test in ['Female']:# genders to be considered in the test and selection of top-performing models
            for include_age_biased_models in [True]: # whether or not to include age-biased corrected models among brain clock modeling alternatives
                for p_value_elimination in [True]: # whether or not to eliminate OLS models that do not clear coefficients t-test
                  output = tm.output_generator(test_data, binary_cases_test, binary_columns_test, age_limits = age_limits, predictors_test = full_set_of_predictors,
                                                gender_test=gender_test, save_analyzed_cases = True, p_value_elimination = p_value_elimination, include_age_biased_models = include_age_biased_models,
                                                pareto_feature_cases = [['MAE', 'MAE_max_bin','auroc_CN_noCN_orx'], ['MAE_bounded', 'MAE_max_bin_bounded','auroc_CN_noCN_orx_bounded']],
                                                age_bound = [55,80], db = db)
                  output.test_models()